<a href="https://colab.research.google.com/github/dohyun-kim92/TIL/blob/master/Ngram%ED%9B%84_NBC%2C_polarityScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이 파일은 다음과 같은걸 했습니다.
- json 파일 불러오기 ( 아주 간단하게 pd.read_json )  
- 행오름차순 처리 axis = 0  ( axis = 1 은 열 오름차순 처리 )  
- 행(열) 삭제  
- 행(열) 순서 변경  
- NaN값 제거  
- Groupby로 데이터 확인 df.groupby(['date','ud'])['ud'].count()
- 컬럼명 변경 
- 리스트 맨앞 '[', 맨뒤']' 제거  (join, lambda, map 사용) 
- 판다스 lambda, map 사용
- 판다스 리인덱싱 df.reset_index(drop=True, inplace=True)
- 파이프라인에서 하나 지정해서 뽑아서 쓰기 (named_steps['clf'])  
clf = text_clf.named_steps['clf']   
  ( 전처리 끝~ NBC 시작 ) 
  
- NBC

In [ ]:
import requests
import csv
import math
import json
from bs4 import BeautifulSoup
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



In [ ]:
# ngram 데이터 받아오기
df = pd.read_json(r'C:\Users\dhsmi\Downloads\final_ngram.json')
df


,date,ngram
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설..."
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융..."
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]"
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정..."
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수..."
...,...,...
305067,2020-07-15,"[실망/NNG, 중단/NNG]"
305068,2020-07-15,"[경기/NNG;침체/NNG, 완화/NNG]"
305069,2020-07-15,"[코픽스/NNG;떨어/VV, 돈/NNG;확보/NNG, 하락/NNG, 내림/NNG]"
305070,2020-07-15,"[저축성/NNG;예금/NNG;늘/VV, 경기/NNG;악화/NNG, 통화/NNG;급증..."


In [ ]:
# 회사채 데이터 받아오기
cb = pd.read_json(r'C:\Users\dhsmi\Downloads\labeled_cb_rate.json')
cb

,date,rate,ud
0,2005-04-01,4.33,-1
1,2005-04-02,4.33,-1
2,2005-04-03,4.33,-1
3,2005-04-04,4.35,-1
4,2005-04-05,4.35,-1
...,...,...,...
5538,2020-05-30,2.15,1
5539,2020-05-31,2.15,1
5540,2020-06-01,2.16,1
5541,2020-06-02,2.18,1


In [ ]:
# ngram & 회사체 합체 및 행오름차순 처리 axis = 0  ( axis = 1 은 열 오름차순 처리 )
cb_mg = df.merge(cb , on = 'date', how = 'left')
print(type(cb_mg))

cb_mg = cb_mg.sort_values(by=['date']) #axis = 0 이면 행을 기준으로, axis = 1이면 열을 기준으로 오름차순 
type(cb_mg['ngram'][0])

# NaN값 제거 -> 인덱스도 지워지니까 인덱스 reset 
cb_mg = cb_mg.dropna(subset = ['ngram'])
cb_mg = cb_mg.dropna(subset = ['rate'])
cb_mg


# 처음에 저장할 때 콤마로 조인하면 ngram의 [] 대괄호 지워짐 ..
# 여기서 지우려면 람다펑션 replace 쓰면..
cb_mg["ngram"][0]
cb_mg
cb_mg.reset_index(drop=True, inplace=True)


<class 'pandas.core.frame.DataFrame'>


In [ ]:
# 리스트 맨앞 '[', 맨뒤']' 제거  (join, lambda, map 사용)
cb_mg['ngram'] = list(map(lambda i : ','.join(i), cb_mg['ngram']))
cb_mg
cb_mg.to_json("nn.json")


In [ ]:

# cb_mg[cb_mg['ud']]==-1

In [ ]:
# 각 class 수 아래서 multiNB 수 잘나왔나 확인하기 위함 
test_tmp = []
for i in range(len(cb_mg)):
    test_tmp.append(cb_mg['ud'].iloc[i])
test_tmp

[-1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,

In [ ]:
# 각 class 수 아래서 multiNB 수 잘나왔나 확인하기 위함 
cb_mg.iloc[0]

date                                2005-05-01 00:00:00
ngram    수입/NNG;증대/NNG,실적/NNG;높/VV,하락/NNG,위험/NNG,자신/NNG
rate                                               4.18
ud                                                   -1
Name: 0, dtype: object

In [ ]:
# 5월전 행 삭제 , ngram = NaN 삭제

cb_mg = cb_mg[cb_mg.date >= '2005-05-01'] 
# cb_mg = cb_mg[cb_mg.date != '[]']
# cb_mg['ngram'][30]



In [ ]:
# 컬럼 순서 변경 

cb_mg = cb_mg[['date', 'rate', 'ngram', 'ud']]
cb_mg

# type(cb_mg["ngram"][0])

len(cb_mg["ngram"][0])
cb_mg

,date,rate,ngram,ud
0,2005-05-01,4.18,"수입/NNG;증대/NNG,실적/NNG;높/VV,하락/NNG,위험/NNG,자신/NNG",-1.0
1,2005-05-01,4.18,"기업/NNG;수익성/NNG;악화/NNG;우려/NNG,경제/NNG;지표/NNG;성장/...",-1.0
2,2005-05-01,4.18,"인플레이션/NNG;압력/NNG;억제/NNG,금리/NNG;인상/NNG;긴축/NNG,지...",-1.0
3,2005-05-01,4.18,"대출/NNG;연체율/NNG;오르/VV,대출/NNG;연체율/NNG;높/VA,대출심사/...",-1.0
4,2005-05-01,4.18,"무수익여신/NNG;증가/NNG,수익성/NNG;악화/NNG,악화/NNG;조짐/NNG,...",-1.0
...,...,...,...,...
291601,2020-06-03,2.19,"대부업/NNG;고금리/NNG,최대/NNG",1.0
291602,2020-06-03,2.19,"매출/NNG;감소/NNG,금리/NNG;낮/VA,타격/NNG,최저/NNG,최대/NNG",1.0
291603,2020-06-03,2.19,"안정/NNG,유예/NNG,최대/NNG",1.0
291604,2020-06-03,2.19,"금융채/NNG;발행/NNG;급증/NNG,금리/NNG;비교/NNG;낮/VA",1.0


## NBC
-1은 dovish 1은 hokish // ngram(x)랑 y(ud) 사용 ...  date랑 rate 는 ngram, ud 뽑는데 쓴거라 이제 필요없음 ~

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import numpy as np

# train_test_split? 

In [ ]:
def com_split(x):
    return x.split(",")


X_train, X_test, y_train, y_test = train_test_split(cb_mg['ngram'], cb_mg['ud'], train_size= 0.9, shuffle = True)

text_clf = Pipeline([   ('cv',CountVectorizer(tokenizer= com_split)), # split 뒤에 (x) 써줄필요 없음 
#                         ('tfidfV', TfidfVectorizer()), -> 논문에 따르면 tfidf 써줄 필요 없음 
                        ('clf', MultinomialNB())
])                                                      # 아래꺼안쓰는 이유는 우리는 ngram 해놨으니까~ tfidfvectorizer로씀 
                                                        # tfidftransformer는 countvect에서 넘어오는거라 안됨 


# ud[1].class_count_ , cb_mg['ud'].feature_count_
text_clf.fit(X_train, y_train) # 처음에 에러떴던 이유는 ngram에서 대괄호('[]')를 제거하지 않았기때문 


clf = text_clf.named_steps['clf']
cv = text_clf.named_steps['cv']
print(clf.class_count_, "\n", clf.feature_count_) 
print("{} : {}".format('clf.classes_',clf.classes_))

print(sum(clf.feature_count_[0]))
print(sum(clf.feature_count_[1]))
print(sum(clf.feature_count_[2]))

# text_clf
clf.feature_count_.shape
cv.vocabulary_
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

[114359.  52496.  95590.] 
 [[ 1.  3.  3. ... 28.  4.  4.]
 [ 2.  3.  3. ... 11.  5.  1.]
 [ 0.  6.  2. ... 14.  7.  4.]]
clf.classes_ : [-1.  0.  1.]
1247105.0
579503.0
1041530.0


0.4900723569150578

In [ ]:
cv.get_feature_names()

['bei/nng;급락/nng',
 'bei/nng;낮/va',
 'bei/nng;높/va',
 'bei/nng;상승/nng',
 'bei/nng;상회/nng',
 'bei/nng;오르/vv',
 'bei/nng;저점/nng',
 'bei/nng;최저/nng',
 'bei/nng;축소/nng',
 'bei/nng;하락/nng',
 'bei/nng;확대/nng',
 'bis/nng;경고/nng',
 'bis/nng;낮/va',
 'bis/nng;떨어/vv',
 'bis/nng;비율/nng;개선/nng',
 'bis/nng;비율/nng;낮/va',
 'bis/nng;비율/nng;높/va',
 'bis/nng;비율/nng;높/vv',
 'bis/nng;비율/nng;떨어/vv',
 'bis/nng;비율/nng;상승/nng',
 'bis/nng;비율/nng;양호/nng',
 'bis/nng;비율/nng;오르/vv',
 'bis/nng;비율/nng;하락/nng',
 'bis/nng;위험가중치/nng;상향/nng',
 'bis/nng;자기자본비율/nng;개선/nng',
 'bis/nng;자기자본비율/nng;낮/va',
 'bis/nng;자기자본비율/nng;높/va',
 'bis/nng;자기자본비율/nng;높/vv',
 'bis/nng;자기자본비율/nng;떨어/vv',
 'bis/nng;자기자본비율/nng;상승/nng',
 'bis/nng;자기자본비율/nng;오르/vv',
 'bis/nng;자기자본비율/nng;하락/nng',
 'bis/nng;하락/nng',
 'cac40/nng;오르/vv;지수/nng;상승/nng',
 'cac40/nng;지수/nng;상승/nng',
 'cac40/nng;지수/nng;오르/vv',
 'cac40/nng;지수/nng;하락/nng',
 'cd/nng;cp/nng;금리/nng;상승/nng',
 'cd/nng;cp/nng;금리/nng;하락/nng',
 'cd/nng;cp/nng;상승/nng',
 'cd/nng;cp/nng;오르/vv',
 'cd/n

In [ ]:
cv.vocabulary_

{'단기/nng;금리/nng;상승/nng': 21672,
 '금리/nng;상승/nng;약세/nng': 14121,
 '금리/nng;낙폭/nng;크/vv': 13116,
 '금리/nng;급락/nng': 13054,
 '국고채/nng;높/va': 10166,
 '부담/nng': 27805,
 '경제/nng;지표/nng;좋/va': 7889,
 '좋/va;정책/nng;아니/vcn': 53766,
 '실업률/nng;상승/nng': 37069,
 '닷컴/nng;거품/nng': 22586,
 '경제/nng;성장/nng': 7025,
 '물가/nng;잡/vv': 26331,
 '통제/nng': 59559,
 '세계/nng;금융시장/nng;불안/nng': 32741,
 '금융시장/nng;불안/nng;우려/nng': 17898,
 '경상/nng;수지/nng;개선/nng': 6557,
 '건전성/nng;의구심/nng': 4710,
 '경제/nng;회복/nng': 8138,
 '긴축/nng': 19913,
 'ecb/nng;금리/nng;인하/nng': 453,
 'ecb/nng;금리/nng;내리/vv': 417,
 '국채선물/nng;하락/nng': 11756,
 '가격/nng;부담/nng': 2683,
 '하단/nng': 61438,
 '금리/nng;인하/nng;경기/nng;침체/nng': 16293,
 '신용위기/nng;경기/nng;둔화/nng': 36467,
 '대출/nng;연체율/nng;상승/nng': 23434,
 'fed/nng;금리/nng;인하/nng': 1021,
 '경기/nng;둔화/nng;우려/nng': 5117,
 '공포/nng;심리/nng;완화/nng': 9381,
 '경기/nng;침체/nng;막/vv': 5976,
 '탈출/nng;성공/nng': 59356,
 '위기/nng;확산/nng': 43324,
 '예상/nng;상회/nng': 40145,
 '악화/nng;막/vv': 37992,
 '위험/nng': 43491,
 '지표/nng;부진/nng;경기/nng

# 위에까진 완료 
### ※ ploarity score 나오는데시간 너무오래걸리므로 for문 말고 Pandas 써서 해결할 것

In [ ]:
# Unique n_gram 수 확인 
import itertools
temp = list(map(lambda i : i.split(','), X_train))
len(set(list(itertools.chain(*temp))))


df_ps = pd.DataFrame(clf.feature_count_)
# text_clf
df_ps
df_ps.T

# 컬럼명 변경
# cb = cb.rename({'일자':'date' , '국고채권\n(10년)':'10rate' }, axis ='columns')
df_ps = df_ps.T.rename({0:-1, 1:0, 2:1}, axis = 'columns' )
df_ps
df_ps["sum(-1)"] = sum(df_ps[-1])
df_ps["sum(1)"] = sum(df_ps[1])
df_ps["-1/sum(-1)"] = df_ps[-1]/sum(df_ps[-1]) # ?? 값이 이상함..
df_ps["1/sum(1)"] = df_ps[1]/sum(df_ps[1])
df_ps
df_ps["p_score"] = df_ps["-1/sum(-1)"]/ df_ps["1/sum(1)"]
df_ps["polarity"] = None
# 판다스 lambda, map 사용 
df_ps.polarity = df_ps.p_score.map(lambda x : "H" if x >= 1.3 else("D" if x< 10/13 else "N"))

df_ps['ngram'] = None
df_ps
df_ps.ngram = cv.get_feature_names() # 이걸써야 인덱스랑 똑같이감 
# df_ps.ngram = cv.vocabulary_m
df_ps
# df_ps.info

,-1,0,1,sum(-1),sum(1),-1/sum(-1),1/sum(1),p_score,polarity,ngram
0,1.0,2.0,1.0,1247618.0,1040140.0,8.015274e-07,9.614090e-07,0.833701,N,bei/nng;급락/nng
1,3.0,3.0,6.0,1247618.0,1040140.0,2.404582e-06,5.768454e-06,0.416850,D,bei/nng;낮/va
2,3.0,2.0,2.0,1247618.0,1040140.0,2.404582e-06,1.922818e-06,1.250551,N,bei/nng;높/va
3,18.0,11.0,23.0,1247618.0,1040140.0,1.442749e-05,2.211241e-05,0.652461,D,bei/nng;상승/nng
4,1.0,0.0,1.0,1247618.0,1040140.0,8.015274e-07,9.614090e-07,0.833701,N,bei/nng;상회/nng
...,...,...,...,...,...,...,...,...,...,...
64962,7.0,4.0,9.0,1247618.0,1040140.0,5.610692e-06,8.652681e-06,0.648434,D,흥행/nng;부진/nng
64963,75.0,44.0,59.0,1247618.0,1040140.0,6.011455e-05,5.672313e-05,1.059789,N,흥행/nng;성공/nng
64964,26.0,10.0,15.0,1247618.0,1040140.0,2.083971e-05,1.442114e-05,1.445081,H,흥행/nng;실패/nng
64965,5.0,3.0,8.0,1247618.0,1040140.0,4.007637e-06,7.691272e-06,0.521063,D,희박/nng;금리/nng;인상/nng


In [ ]:
cv.vocabulary_

{'단기/nng;금리/nng;상승/nng': 21672,
 '금리/nng;상승/nng;약세/nng': 14121,
 '금리/nng;낙폭/nng;크/vv': 13116,
 '금리/nng;급락/nng': 13054,
 '국고채/nng;높/va': 10166,
 '부담/nng': 27805,
 '경제/nng;지표/nng;좋/va': 7889,
 '좋/va;정책/nng;아니/vcn': 53766,
 '실업률/nng;상승/nng': 37069,
 '닷컴/nng;거품/nng': 22586,
 '경제/nng;성장/nng': 7025,
 '물가/nng;잡/vv': 26331,
 '통제/nng': 59559,
 '세계/nng;금융시장/nng;불안/nng': 32741,
 '금융시장/nng;불안/nng;우려/nng': 17898,
 '경상/nng;수지/nng;개선/nng': 6557,
 '건전성/nng;의구심/nng': 4710,
 '경제/nng;회복/nng': 8138,
 '긴축/nng': 19913,
 'ecb/nng;금리/nng;인하/nng': 453,
 'ecb/nng;금리/nng;내리/vv': 417,
 '국채선물/nng;하락/nng': 11756,
 '가격/nng;부담/nng': 2683,
 '하단/nng': 61438,
 '금리/nng;인하/nng;경기/nng;침체/nng': 16293,
 '신용위기/nng;경기/nng;둔화/nng': 36467,
 '대출/nng;연체율/nng;상승/nng': 23434,
 'fed/nng;금리/nng;인하/nng': 1021,
 '경기/nng;둔화/nng;우려/nng': 5117,
 '공포/nng;심리/nng;완화/nng': 9381,
 '경기/nng;침체/nng;막/vv': 5976,
 '탈출/nng;성공/nng': 59356,
 '위기/nng;확산/nng': 43324,
 '예상/nng;상회/nng': 40145,
 '악화/nng;막/vv': 37992,
 '위험/nng': 43491,
 '지표/nng;부진/nng;경기/nng

In [ ]:
df_iter30 = pd.DataFrame()
df_iter30.ngram = None


# train_size를 1로놔서 전체 ngram 생성해서 인덱스로 놓기 
X_train, y_train = cb_mg['ngram'], cb_mg['ud']
text_clf = Pipeline([   ('cv',CountVectorizer(tokenizer= com_split)),
                        ('clf', MultinomialNB())
                    ])                             
text_clf.fit(X_train, y_train) 
clf = text_clf.named_steps['clf']
cv = text_clf.named_steps['cv']

cv.get_feature_names()

['bei/nng;급락/nng',
 'bei/nng;낮/va',
 'bei/nng;높/va',
 'bei/nng;상승/nng',
 'bei/nng;상회/nng',
 'bei/nng;스프레드/nng;상승/nng',
 'bei/nng;오르/vv',
 'bei/nng;저점/nng',
 'bei/nng;최저/nng',
 'bei/nng;축소/nng',
 'bei/nng;하락/nng',
 'bei/nng;확대/nng',
 'bis/nng;경고/nng',
 'bis/nng;낮/va',
 'bis/nng;떨어/vv',
 'bis/nng;비율/nng;개선/nng',
 'bis/nng;비율/nng;낮/va',
 'bis/nng;비율/nng;높/va',
 'bis/nng;비율/nng;높/vv',
 'bis/nng;비율/nng;떨어/vv',
 'bis/nng;비율/nng;상승/nng',
 'bis/nng;비율/nng;양호/nng',
 'bis/nng;비율/nng;오르/vv',
 'bis/nng;비율/nng;하락/nng',
 'bis/nng;위험가중치/nng;상향/nng',
 'bis/nng;자기자본비율/nng;개선/nng',
 'bis/nng;자기자본비율/nng;낮/va',
 'bis/nng;자기자본비율/nng;높/va',
 'bis/nng;자기자본비율/nng;높/vv',
 'bis/nng;자기자본비율/nng;떨어/vv',
 'bis/nng;자기자본비율/nng;상승/nng',
 'bis/nng;자기자본비율/nng;오르/vv',
 'bis/nng;자기자본비율/nng;하락/nng',
 'bis/nng;하락/nng',
 'cac40/nng;오르/vv;지수/nng;상승/nng',
 'cac40/nng;지수/nng;상승/nng',
 'cac40/nng;지수/nng;오르/vv',
 'cac40/nng;지수/nng;하락/nng',
 'cd/nng;cp/nng;금리/nng;상승/nng',
 'cd/nng;cp/nng;금리/nng;하락/nng',
 'cd/nng;cp/nng;상승/nng',
 '

In [ ]:
df_iter30['ngram'] = cv.get_feature_names()

In [ ]:
#df_iter30['ngrams'] = ''
df_iter30

,ngram
0,bei/nng;급락/nng
1,bei/nng;낮/va
2,bei/nng;높/va
3,bei/nng;상승/nng
4,bei/nng;상회/nng
...,...
65431,흥행/nng;부진/nng
65432,흥행/nng;성공/nng
65433,흥행/nng;실패/nng
65434,희박/nng;금리/nng;인상/nng


In [ ]:
score_dic = {} # 스코어는 리스트로 ..  -> 위에꺼 다지우고 다시짜라 .. 한번에 다넣고 처리하렿지말구 나오면 넣는식으로.. 형태를 다시 

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(cb_mg['ngram'], cb_mg['ud'], train_size= 0.9, shuffle = True)

    text_clf = Pipeline([   ('cv',CountVectorizer(tokenizer= com_split)), 
                            ('clf', MultinomialNB())
    ])                                                      
    text_clf.fit(X_train, y_train) 

    clf = text_clf.named_steps['clf']
    cv = text_clf.named_steps['cv']


    df_ps = pd.DataFrame(clf.feature_count_)
    df_ps = df_ps.T.rename({0:-1, 1:0, 2:1}, axis = 'columns' )
    df_ps["sum(-1)"] = sum(df_ps[-1])
    df_ps["sum(1)"] = sum(df_ps[1])
    df_ps["-1/sum(-1)"] = df_ps[-1]/sum(df_ps[-1]) 
    df_ps["1/sum(1)"] = df_ps[1]/sum(df_ps[1])
    df_ps["p_score"] = df_ps["-1/sum(-1)"]/ df_ps["1/sum(1)"]
    df_ps['ngram'] = None
    df_ps.ngram = cv.get_feature_names()

    score_dic[key].append(cv.get_feature_names())
    ps_iter30["ngram"].append(cv.get_feature_names())
    ps_iter30["p_score"].append(df_ps["p_score"])
    
    df_iter30 = df_iter30.merge(df_ps[['ngram','p_score']], on = 'ngram' , how = 'left')
#     if df_iter30.ngram in df_ps.ngram: # merge를 써볼...가..? cf) cb_all = date_only.merge(cb , on = 'date', how = 'left')
#         df_iter30[i] = df_ps.p_score
# 1번 이터네 넣을거.. [ngram], [p_score]


In [ ]:
df_iter30

,ngram,p_score_x,p_score_y,p_score_x,p_score_y,p_score_x,p_score_y,p_score_x,p_score_y,p_score_x,...,p_score_x,p_score_y,p_score_x,p_score_y,p_score_x,p_score_y,p_score_x,p_score_y,p_score_x,p_score_y
0,bei/nng;급락/nng,1.230623,2.494204,1.112567,2.499613,0.000000,inf,3.753629,0.000000,0.556032,...,1.340300,inf,0.634052,0.830029,NaN,inf,1.005553,NaN,0.000000,inf
1,bei/nng;낮/va,3.329921,0.475087,0.000000,2.499613,1.510929,0.000000,2.383257,1.486835,1.158400,...,4.188436,2.083327,2.502836,0.276676,0.740352,0.833960,0.833172,0.623596,0.000000,1.941129
2,bei/nng;높/va,3.746161,1.478047,1.251638,1.562258,0.836822,1.462748,1.167796,0.520392,0.758225,...,0.558458,1.249996,1.311009,0.830029,0.277632,inf,inf,0.475121,0.312203,0.554608
3,bei/nng;상승/nng,0.416240,0.831401,5.006551,0.624903,inf,2.507568,0.166828,0.763242,0.000000,...,inf,0.499998,2.085696,0.860771,0.910981,0.758146,1.041466,7.483149,0.208136,2.079781
4,bei/nng;상회/nng,0.104060,0.831401,1.112567,0.833204,5.857756,1.985158,1.112186,0.118947,2.502144,...,1.256531,0.624998,1.668557,6.640233,2.915138,inf,inf,1.306582,inf,2.079781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65431,흥행/nng;부진/nng,0.000000,inf,0.476814,4.166022,0.836822,0.835856,1.251210,0.208157,0.705733,...,1.090438,0.777775,0.641753,0.664023,3.053954,1.459430,0.772420,0.534511,0.277514,0.623934
65432,흥행/nng;성공/nng,0.416240,2.494204,inf,1.203517,0.669458,0.000000,0.000000,0.475787,inf,...,0.762390,inf,0.856365,1.162041,0.832897,1.667920,0.000000,0.623596,0.416271,0.831912
65433,흥행/nng;실패/nng,0.520300,4.988409,8.344252,0.416602,0.228224,1.838883,1.668280,1.170179,inf,...,0.000000,0.499998,0.000000,0.415015,1.025103,0.833960,0.499903,0.692884,0.999050,0.499147
65434,희박/nng;금리/nng;인상/nng,0.739982,2.494204,0.667540,1.110939,inf,2.507568,0.667312,2.497883,1.167667,...,0.670150,4.166653,inf,inf,1.665793,inf,0.624879,1.351124,0.832542,1.164677


In [ ]:
score_dic

{}

In [ ]:
score_dic['금리'] = [0.5]

In [ ]:
score_dic['금리'].append(0.03)

In [ ]:
df_ps['ngram']


0                  금리/nng;인하/nng;예상/nng;작/va
1                      금리/nng;마이너스/nng;내리/vv
2                              지수/nng;하락/nng
3                                     실망/nng
4                                     최대/nng
                        ...                 
64993                     물가/nng;우려/nng;않/vx
64994            선행/nng;지수/nng;고점/nng;하락/nng
64995                         안정화/nng;강세/nng
64996    fed/nng;금리/nng;인상/nng;단행/nng;강세/nng
64997           fed/nng;금리/nng;결정/nng;하락/nng
Name: ngram, Length: 64998, dtype: object

## 아래 코드는 pandas 쓰는 것보다 시간이 너무 많이 걸려서 주석처리함

In [ ]:


# polarity score 계산  -> 로그 쓸필요없음 .. 어차피 분자가 0 되는건 필요없는것으로 간주하게됨  -> if문으로 해결 
# p_score = []  

# for i in range(len(clf.feature_count_[0])):
#     if clf.feature_count_[0][i] !=0 and clf.feature_count_[2][i] !=0 :
#         p_score.append(
#                       (clf.feature_count_[0][i] / sum(clf.feature_count_[0])) 
#             /
#             (clf.feature_count_[2][i] / sum(clf.feature_count_[2]))
#                       )

# p_score


# 이거 막으려면 log 써라  분모가 0 이어서... warning 뜸 
# 시간 tqdm ... 

# 위의 코드 polarity iter =30 돌리기 
### 30번 돌린후 mean값으로 각 ngram의 polarity 선정